In [23]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def hello(self):
        return f"Hello, I'm {self.name}"

chris = Person('Chris')  # "chris" is an instance of Person

In [24]:
chris

In [25]:
chris.hello()

"Hello, I'm Chris"

In [3]:
type(chris)  # The type of chris is Person

__main__.Person

In [7]:
# Well, what's the type of Person?
type(Person)

type

##### ?!?!?

`type` is a class and a built in!

- A class is a blueprint for an instance of the class.
- A metaclass is a blueprint for an instance of "type", ie. a class. A metaclass is a class-factory.

In [11]:
# Is there a blueprint for an instance of metaclasses? Yes. It's a metaclass :)
type(type)

type

In [ ]:
# When is a metaclass used?
class Person:  
    def __init__(self, name):
        self.name = name
# <<< Right here. Right after all of the class has been read in.

The metaclass for a class is invoked when a class definition is read in by Python. Python creates a new class as soon as it sees this new class definition. To do so, Python calls the metaclass.

Python searches through parents classes to see what metaclass to use. Metaclasses are passed down via inheritance. Often, this metaclass ends up being `type`. So how is `type` called?

In [32]:
def __init__(self, name):
    self.name = name

def hello(self):
    return f"Hello, I'm {self.name}"

classname = 'Person'
bases = ()
methods = {
    '__init__': __init__,
    'hello': hello,
}

# We programmatically create the class
type(classname, bases, methods)

__main__.Person

In [34]:
janet = type(classname, bases, methods)('Janet')
janet

In [35]:
janet.hello()

"Hello, I'm Janet"

*What do I do with this?*

- Build custom classes on the fly
- Create a class with a different metaclass

In [43]:
class CustomType(type):
    def __new__(cls, name, bases, dict_):
        print(f'Just started creating new class "{name}"!')
        new_cls = super().__new__(cls, name, bases, dict_)
        print(f'All done creating new class {new_cls}')
        return new_cls

class Person(metaclass=CustomType):  # Override 'type' as metaclass
    pass

Just started creating new class "Person"!
All done creating new class <class '__main__.Person'>


In [44]:
class Parent(Person):
    pass

Just started creating new class "Parent"!
All done creating new class <class '__main__.Parent'>


In [45]:
class Child(Person):
    pass

Just started creating new class "Child"!
All done creating new class <class '__main__.Child'>


In [47]:
registry = set()

class RegisteredClassType(type):
    def __new__(cls, name, bases, dict_):
        new_cls = super().__new__(cls, name, bases, dict_)
        # We don't want to register the parent class that all other
        # registered classes will inherit from.
        if name != 'RegisteredClass':
            registry.add(new_cls)
        return new_cls

class RegisteredClass(metaclass=RegisteredClassType):
    pass

class A(RegisteredClass):
    pass

class B(RegisteredClass):
    pass

class Panda(RegisteredClass):
    pass

registry

{__main__.A, __main__.B, __main__.Panda}

In [79]:
class PreparedType(type):
    def __prepare__(name, bases, **kwargs) -> dict:
        """
        Return a dict that will hold all the attributes of 
        the class keyed by the name of the attribute. The
        dict can be empty. It's just a starting point that
        will hold all attributes to come.
        
        This method is called before __new__ and its result
        is passed to __new__ as the third argument "dict_".
        """
        def my_property(func):
            def decorated(*a, **kw):
                print('Calling method!')
                return func(*a, **kw)
            return decorated
        
        def not_available(func):
            pass
        
        return {
            'my_property': my_property,
        }


class PreparedClass(metaclass=PreparedType):
    pass


class MyClass(PreparedClass):
    
    @my_property
    def run(self):
        print('Yeah!')
      
    # The following lines will raise a NameError when uncommented.
    # @not_available
    # def not_going_to_work(self):
    #     print('Nope')

MyClass().run()

Calling method!
Yeah!


> Metaclasses are deeper magic that 99% of users should never worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why).
-- Tim Peters

